In [1]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [2]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) != 0 and float(p[11]) != 0):
                return p

In [3]:
texilinesCorrect = rdd.filter(correctRows)

In [4]:
n = texilinesCorrect.count()

In [5]:
distance_money = texilinesCorrect.map(lambda x: (1, (float(x[5]), float(x[11]))))

In [6]:
distance_money.take(1)

[(1, (0.44, 3.5))]

In [7]:
def getSumPart(x):
    partOfDm = x[1][0] * (x[1][1] - (m * x[1][0] + b))
    partOfDb = (x[1][1] - (m * x[1][0] + b))
    return (x[0], (partOfDm, partOfDb))

In [8]:
sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))

In [12]:
import numpy as np
import time 

In [13]:
def gradientDescent(learningRate = 0.001, num_iteration = 100):
    prev_m = 0.1
    prev_b = 0.1
    count = 0
    precision = 0.00000001
    prev_stepsize = 1
    while(prev_stepsize > precision and num_iteration > count):
        
        start = time.time()
        
        def getSumPart(x):
            partOfDm = - x[1][0] * (x[1][1] - (prev_m * x[1][0] + prev_b))
            partOfDb = - (x[1][1] - (prev_m * x[1][0] + prev_b))
            return (x[0], (partOfDm, partOfDb))
        
        sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))\
                            .collect()
        
        dm = (2/n) * sumInGradient[0][1][0]
        db = (2/n) * sumInGradient[0][1][1]
        
        cur_m = prev_m - (learningRate * dm)
        cur_b = prev_b - (learningRate * db)
        
        count += 1
        
        prev_stepsize = np.mean([abs(prev_m - cur_m), abs(prev_b - cur_b)])
        
        prev_m = cur_m
        prev_b = cur_b
        
        stop = time.time()
        duration = stop - start
        
        print("Iteration: ", count)
        print("m: ", cur_m, "b: ", cur_b)
        print("Duration", duration)
        
        
    return cur_m, cur_b

In [14]:
m, b = gradientDescent()

Iteration:  1
m:  0.22783430859133066 b:  0.12283392434996293
Duration 15.482970952987671
Iteration:  2
m:  0.3503532687352195 b:  0.14487968922509267
Duration 16.232582807540894
Iteration:  3
m:  0.4677769558112367 b:  0.16616974373392746
Duration 15.851143836975098
Iteration:  4
m:  0.5803163343187325 b:  0.18673519383741136
Duration 16.60724401473999
Iteration:  5
m:  0.6881736350562151 b:  0.20660585795330697
Duration 14.35943603515625
Iteration:  6
m:  0.7915427166856943 b:  0.2258103202586679
Duration 17.50995111465454
Iteration:  7
m:  0.8906094123290277 b:  0.244375981785639
Duration 17.71121382713318
Iteration:  8
m:  0.9855518618153362 b:  0.2623291094019777
Duration 16.632179021835327
Iteration:  9
m:  1.0765408301738448 b:  0.2796948827638394
Duration 16.23905110359192
Iteration:  10
m:  1.163740012941702 b:  0.29649743932473255
Duration 15.583604335784912
Iteration:  11
m:  1.2473063288322126 b:  0.3127599174812188
Duration 15.447594165802002
Iteration:  12
m:  1.327390200

KeyboardInterrupt: 